# Spectral Event Analysis Tutorial

This tutorial is a hands-on introduction to using the [SpectralEvents toolbox](https://github.com/jonescompneurolab/SpectralEvents), **a collection of functions designed to help researchers characterize high-amplitude peaks in the spectral representation of continuous neural signals**. Here, we'll load and analyze data that is distributed with the toolbox, as described in [Shin et al. (2017)](https://doi.org/10.7554/eLife.29086).

First, we'll import some dependencies. Note that `seaborn` is not required, but is added to improve plot styling.

In [ ]:
%matplotlib widget

import sys
import os.path as op
from glob import glob

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

Now let's import the SpectralEvents toolbox module. You might need to modify the path to this module so that your Python interpreter knows where to find it.

In [ ]:
# set path to SpectralEvents if necessary
#sys.path.append('/home/ryan/SpectralEvents')
import spectralevents as se

In [ ]:
# dataset parameters
data_dir = op.join(op.dirname(se.__file__), 'data') 
subj_ids = [str(id) for id in range(1, 10 + 1)]  # subject IDs 1-10

n_subjs = len(subj_ids)  # number of subjects
n_trials = 200           # number of trials per subject
n_times = 600            # number of time samples per trial
samp_freq = 600          # sampling rate (Hz)

In [ ]:
# load data
hit_trials = list()
data = list()
for id_idx, id in enumerate(subj_ids):
    fname = op.join(data_dir, 'prestim_humandetection_600hzMEG_subject' + id + '.mat')
    raw_data = loadmat(fname)
    hit_trials.append(np.nonzero(raw_data['YorN'].squeeze())[0])  # indices for trials where the stimulus was detected
    data.append(raw_data['prestim_raw_yes_no'])  # MEG time series (trials x samples)

Once you've loaded the data, make sure you understand how it's formatted. There
are 10 subjects, each with 100 detected (i.e., *hit*) trials and 100 undetected (i.e., *miss*) trials. How might you go about selecting only the *hit* trials from the 1st subject?

In [ ]:
# investigate data structure: # trials x time for Subj 1
print(data[0].shape)

## Identifying events in single-subject data
Let's find the spectral events across trial conditions in a single subject,
after which we'll expand the analysis to include all 10 subjects.

In [ ]:
subj_data = data[0]

We've loaded the data and must now set the conditions governing Spectral Event detection:
- `freqs`, the frequency values over which you will calculate your **t**ime-**f**requency **r**esponse (TFR)
- `times`, the time values at which your signal was sampled relative to each epoch or trial
- `event_band`, the bounds of the frequency band in which you will look for Spectral Events
- `thresh_FOM`, the factor-of-the-median threshold that will be used to find suprathreshold spectral power in each frequency bin across the spectrogram(s)

The general workflow for event detection follows this progression: **TIMESERIES** -> **TFR** -> **SPECTRAL EVENTS**

In [ ]:
# set parameters
freqs = list(range(1, 60 + 1))   # fequency values (Hz) over which to calculate TFR
times = np.arange(n_times) / samp_freq  # seconds
event_band = [15, 29]  # beta band (Hz)
thresh_FOM = 6.  # factor-of-the-median threshold

**Step 1**: **TIMESERIES** -> **TFR**

In [ ]:
# calculate TFR
tfrs = se.tfr(subj_data, freqs, samp_freq)

**Step 2**: **TFR** -> **SPECTRAL EVENTS**

(Note that our band-of-interest, as guided by the literature and discussed in more detail in a few moments, is the **beta band (15-29 Hz)**.)

In [ ]:
# find spectral events!!
spec_events = se.find_events(tfr=tfrs, times=times, freqs=freqs,
                             event_band=event_band, threshold_FOM=thresh_FOM)

`spec_events` is is a nested list (trials x events). Each event is annotated by
dictionary key-value pairs that include `Peak Time`, `Peak Frequency`,
`Peak Power`, `Duration`, `Frequency Span`, amoung a few others.

In [ ]:
print(len(spec_events))  # number of trials
print(len(spec_events[0]))  # number of events in the 1st trial
spec_events[0][0]  # features of the 1st event of the 1st trial

Before we investigate anything else about the events that were detected in the
the data (we'll come back to this shortly), let's see what the average
spectrogram looks like.

In [ ]:
fig = se.plot_avg_spectrogram(tfr=tfrs, times=times, freqs=freqs,
                              event_band=event_band) 

What if we separate between the two experimental conditions, *hit* versus *miss* trials?

In [ ]:
subj_hit_trials = hit_trials[0]
subj_miss_trials = [idx for idx in range(n_trials) if idx not in subj_hit_trials]
vlim = [0, 6.0e-17]

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_hit_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              vlim=vlim)  # note the vlim arguement!

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_miss_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              vlim=vlim)  # note the vlim arguement!

A few observations:

1. It appears that there is **beta** activity across time and trials.

2. This activity is more pronounced in *miss* trials specficially.

If the high beta power is indeed driven by a beta *rhythm*, we should be
able to observe it in individual trials. Let's see what a few individual
trials look like.

In [ ]:
example_trials = [0, 27, 65]
vlim = [0, 7.5e-17]

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_hit_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              timeseries=subj_data[subj_hit_trials],
                              example_epochs=example_trials,  # note the example_epochs arguement!
                              vlim=vlim)

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_miss_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              timeseries=subj_data[subj_miss_trials],
                              example_epochs=example_trials,  # note the example_epochs arguement!
                              vlim=vlim)

Note that the example single-trial signals don't appear to exhibit continuous
beta rhythms; rather, they exhibit spurious "bursts" in beta activity. From now
on, we'll call these *beta events*. Now let's overlay the same spectrograms
with the **beta events** that we detected a few steps earlier.

In [ ]:
hit_spec_events = [trial_events for trial_idx, trial_events in enumerate(spec_events)
                   if trial_idx in subj_hit_trials]
miss_spec_events = [trial_events for trial_idx, trial_events in enumerate(spec_events)
                    if trial_idx in subj_miss_trials]

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_hit_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              spec_events=hit_spec_events,  # note the spec_events argument!
                              timeseries=subj_data[subj_hit_trials],
                              example_epochs=example_trials,
                              show_events=True,  # note the show_events argument!
                              vlim=vlim)

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_miss_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              spec_events=miss_spec_events, # note the spec_events argument!
                              timeseries=subj_data[subj_miss_trials],
                              example_epochs=example_trials,
                              show_events=True,  # note the show_events argument!
                              vlim=vlim)

Indeed, it appears that single trial spectrograms might contain
non-time-locked beta events. Importantly, the *miss* trials appear to have more
of them. To investigate this further we can plot the distribution of spectral
events per trial.

In [ ]:
hit_event_counts = [len(events) for events in hit_spec_events]
miss_event_counts = [len(events) for events in miss_spec_events]

plt.figure()
plt.hist(hit_event_counts, label='hit', bins=8, density=True, alpha=0.7)
plt.hist(miss_event_counts, label='miss', bins=8, density=True, alpha=0.7)
plt.xlabel('events/trial')
plt.ylabel('number of trials')

A few things to consider:

1. What does this tell you about the trial-average spectrogram?

2. Is this trend conserved across subjects?

Before we move on, let's create a function for extracting the trial-average
difference between the spectral event count in *hit* versus *miss* trials.

In [ ]:
def avg_event_count_diff(subj_spec_events, subj_hit_trials):

    # initialize the event count to 0
    hit_event_count = 0
    miss_event_count = 0

    # iterate over trials and add event counts depending on trial type
    for trial_idx, trial_events in enumerate(subj_spec_events):
        if trial_idx in subj_hit_trials:
            hit_event_count += len(trial_events)
        else:
            miss_event_count += len(trial_events)

    return (hit_event_count - miss_event_count) / len(subj_hit_trials)

## Identifying events in multiple subjects
Now let's exand our beta event detection to include data from all 10 subjects.
To do this, we'll create one big loop that iterates over our
**TIMESERIES** -> **TFR** -> **SPECTRAL EVENTS** workflow for each subject.

In [ ]:
# calculate time-frequency response (TFR)
all_tfrs = np.zeros((n_subjs, n_trials, len(freqs), n_times))
all_spec_events = list()
all_hit_miss_count_diff = list()

for subj_idx, subj_data in enumerate(data):

    # calculate TFRs
    tfrs = se.tfr(subj_data, freqs, samp_freq)

    # find events
    spec_events = se.find_events(tfr=tfrs, times=times, freqs=freqs,
                                 event_band=event_band, threshold_FOM=6.)

    # find the average event count difference for this subject
    subj_hit_trials = hit_trials[subj_idx]
    hit_miss_count_diff = avg_event_count_diff(spec_events, subj_hit_trials)
    all_hit_miss_count_diff.append(hit_miss_count_diff)

    # store TFRs and events for later
    all_tfrs[subj_idx, :, :, :] = tfrs
    all_spec_events.append(spec_events)

To test whether beta events are more likely to occur in *miss* trials compared
to *hit* trials across subjects, we can plot the trial-average difference in
the number of detected events for *hit* minus *miss* trials. While a formal
statistical test would require more subjects, this hopefully builds intuition
for how spectral event features can be used to test hypotheses about
differences in spectral events between experimental conditions.

In [ ]:
plt.figure()
plt.violinplot(all_hit_miss_count_diff)
plt.scatter(np.ones((n_subjs,)), all_hit_miss_count_diff)
plt.xticks([])
plt.axhline(0, linestyle=':')
plt.xlabel('difference in # of events')

Other commonly used spectral event features include

- `Peak Power`, an event's power maximum
- `Duration`, an event's width in time
- `Frequency Span`, an event's width in frequency space

These features are left for you to explore further!